# MoTiV Exploration


In [1]:
import os
import pandas as pd
import numpy as np

In [6]:
root_dir = '../data/MoTiV/raw'

In [4]:
['weather_legs.csv',
 'legs.csv',
 'user_details.csv',
 'experience_factors.csv',
 'mots.csv',
 'user_generic_worthwhileness_values.csv',
 'worthwhileness_elements_from_trips.csv',
 'activities.csv',
 'user_specific_worthwhileness_values.csv',
 'weather_raw.csv',
 'legs_coordinates.csv',
 'trips.csv',
 'purposes.csv']

['weather_legs.csv',
 'legs.csv',
 'user_details.csv',
 'experience_factors.csv',
 'mots.csv',
 'user_generic_worthwhileness_values.csv',
 'worthwhileness_elements_from_trips.csv',
 'activities.csv',
 'user_specific_worthwhileness_values.csv',
 'weather_raw.csv',
 'legs_coordinates.csv',
 'trips.csv',
 'purposes.csv']

In [33]:
legs_coordinates = pd.read_csv(f'{root_dir}/legs_coordinates.csv')
legs_coordinates

,legid,start_lat,start_lon,end_lat,end_lon,start_name,start_country,start_class,end_name,end_country,end_class
0,#223:33591,51.100,3.050,51.150,3.050,NaN,NaN,rural,NaN,NaN,rural
1,#224:33591,51.130,3.165,51.130,3.165,Bruges,BEL,sub-urban,Bruges,BEL,sub-urban
2,#225:33590,51.130,3.165,51.130,3.165,Bruges,BEL,sub-urban,Bruges,BEL,sub-urban
3,#222:33592,51.150,3.050,51.150,3.050,NaN,NaN,rural,NaN,NaN,rural
4,#227:9335,51.150,3.050,51.150,3.050,NaN,NaN,rural,NaN,NaN,rural
...,...,...,...,...,...,...,...,...,...,...,...
180508,#224:47071,46.520,6.565,46.520,6.565,Lausanne,CHE,sub-urban,Lausanne,CHE,sub-urban
180509,#225:47061,46.520,6.565,46.545,6.565,Lausanne,CHE,sub-urban,Lausanne,CHE,sub-urban
180510,#222:47083,46.625,6.575,46.520,6.575,Lausanne,CHE,sub-urban,Lausanne,CHE,sub-urban
180511,#223:47078,46.520,6.565,46.520,6.565,Lausanne,CHE,sub-urban,Lausanne,CHE,sub-urban


In [32]:
users = pd.read_csv(f'{root_dir}/user_details.csv')

In [10]:
trips = pd.read_csv(f'{root_dir}/trips.csv')

,tripid,userid,start_date,end_date,average_speed,max_speed,distance,duration,mood_rating,did_you_have_to_arrive,how_often,use_trip_more_for,manual_start,manual_end,validation_date,os,os_version,model
0,#32:1461,o3QNooVleBR9WfbIaCqsaj3DlDl2,1970-01-01 00:25:57.159976,1970-01-01 00:25:57.160092,6.808,20.873,218.0,115283.0,-1.0,-1.0,-1.0,-1.0,True,True,NaT,Android AppVersion 107,28,Xiaomi Mi A2
1,#31:1587,o3QNooVleBR9WfbIaCqsaj3DlDl2,1970-01-01 00:25:57.160122,1970-01-01 00:25:57.160710,1.710,15.467,279.0,587407.0,-1.0,-1.0,-1.0,-1.0,False,True,NaT,Android AppVersion 107,28,Xiaomi Mi A2
2,#31:1449,IGpJc0Az17bxe4MT6pHDx6lQ3LD2,1970-01-01 00:25:57.239471,1970-01-01 00:25:57.239782,4.451,11.279,385.0,311364.0,-1.0,-1.0,-1.0,-1.0,True,True,NaT,Android AppVersion 109,28,HMD Global TA-1012
3,#30:1498,IGpJc0Az17bxe4MT6pHDx6lQ3LD2,1970-01-01 00:25:57.243610,1970-01-01 00:25:57.243809,0.672,23.933,37.0,198138.0,-1.0,-1.0,-1.0,-1.0,True,True,NaT,Android AppVersion 109,28,HMD Global TA-1012
4,#33:1435,IGpJc0Az17bxe4MT6pHDx6lQ3LD2,1970-01-01 00:25:57.244477,1970-01-01 00:25:57.244842,5.727,44.140,580.0,364564.0,-1.0,-1.0,-1.0,-1.0,False,True,NaT,Android AppVersion 109,28,HMD Global TA-1012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67172,#130:20811,8g5HrJBPR6bhIdMr6qln8AAq4pp1,1970-01-01 00:26:14.959017,1970-01-01 00:26:14.960273,16.208,69.205,5655.0,1256016.0,4.0,0.0,1.0,NaN,False,False,1970-01-01 00:26:15.182629,Android AppVersion 171,26,Samsung SM-A320F
67173,#131:20807,8g5HrJBPR6bhIdMr6qln8AAq4pp1,1970-01-01 00:26:14.942673,1970-01-01 00:26:14.944073,14.247,61.984,5539.0,1399630.0,3.0,1.0,1.0,NaN,False,False,1970-01-01 00:26:15.182759,Android AppVersion 171,26,Samsung SM-A320F
67174,#132:20797,8g5HrJBPR6bhIdMr6qln8AAq4pp1,1970-01-01 00:26:14.937811,1970-01-01 00:26:14.939194,16.048,62.317,6167.0,1383441.0,3.0,0.0,1.0,NaN,False,False,1970-01-01 00:26:15.182856,Android AppVersion 171,26,Samsung SM-A320F
67175,#133:20790,8g5HrJBPR6bhIdMr6qln8AAq4pp1,1970-01-01 00:26:14.923772,1970-01-01 00:26:14.925584,37.433,93.522,18842.0,1812088.0,3.0,1.0,1.0,NaN,False,False,1970-01-01 00:26:15.182920,Android AppVersion 171,26,Samsung SM-A320F


In [27]:
user_id_mapping = {id:index for index, id in enumerate(users['userid'])}

In [ ]:
trips_id_mapping = {id:index for index, id in enumerate(trips['userid'])}